In [1]:
import random

In [2]:
class Sentiment: 
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"
    
class Review: 
    def __init__(self, text, score):
        self.text = text 
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <= 2: return Sentiment.NEGATIVE
        elif self.score == 3:  return Sentiment.NEUTRAL
        else: return Sentiment.POSITIVE #Score of 4 or 5

class ReviewContainer: 
    def __init__(self, reviews):
        self.reviews = reviews
    
    def get_text(self):
        return [review.text for review in self.reviews]
    
    def get_sentiment(self):
        return [review.sentiment for review in self.reviews]
    
    def evenly_distribute(self): 
        positive = list(filter(lambda review: review.sentiment == Sentiment.POSITIVE, self.reviews))
        negative = list(filter(lambda review: review.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)

In [3]:
import json

file_name = './data/books_small_10000.json'
reviews = []

with open(file_name) as f: 
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))

reviews[5].text 

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

In [4]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size=0.33, random_state=42)
train_container = ReviewContainer(training)
test_container = ReviewContainer(test)

In [5]:
train_container.evenly_distribute()
train_x = train_container.get_text() 
train_y = train_container.get_sentiment()

test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))

436
436


Bag of words vecorization   

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x)

print(train_x[0])
print(train_x_vectors[0].toarray())

Those of us who have read the whole series will be happy. Could use a little more action maybe with grandma.
[[0. 0. 0. ... 0. 0. 0.]]


In [7]:
test_x[0]

"This was OK. I had problems finishing it and actually read the last chapter at about 65% and decided that it wasn't worth it to finish. That's very unusual for me, but it just wasn't my cup of tea. It started out pretty good and the concept was good, but I became uncomfortable reading it at about 1/4 of the way through."

In [8]:
# Linear SVM
from sklearn import svm 
clf_svm = svm.SVC(kernel='linear')
clf_svm.fit(train_x_vectors, train_y)
clf_svm.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

In [9]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)
clf_dec.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

In [10]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
clf_gnb = DecisionTreeClassifier()
clf_gnb.fit(train_x_vectors, train_y)
clf_gnb.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

In [11]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_y)
clf_log.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

In [12]:
# Evaluation
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
print(clf_gnb.score(test_x_vectors, test_y))
print(clf_log.score(test_x_vectors, test_y))

0.8076923076923077
0.6322115384615384
0.6658653846153846
0.8052884615384616


In [13]:
#F1 Scores 
from sklearn.metrics import f1_score
f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE])

array([0.80582524, 0.80952381])

In [14]:
test_set = ['very fun', 'bad book do not buy', 'horrible waste of time']
new_test = vectorizer.transform(test_set)
clf_svm.predict(new_test)

array(['POSITIVE', 'NEGATIVE', 'NEGATIVE'], dtype='<U8')

In [ ]:
# Tuning our model (with Grid Search)
from sklearn.model_selection import GridSearchCV
parameters = {'kernel': ('linear', 'rbf'), 'C': (1, 4, 8, 16, 32)}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(train_x_vectors, train_y)

In [ ]:
clf.score(test_x_vectors, test_y)

Saving model

In [ ]:
# Save model
import pickle
with open('./models/sentiment_classifier.pkl', 'wb') as f: 
    pickle.dump(clf, f)